# Évaluation et Comparaison des Modèles

Ce notebook réalise l'évaluation complète et la comparaison des modèles entraînés.

## Objectifs
- Charger et comparer tous les modèles entraînés
- Analyser l'impact de la data augmentation
- Comparer les architectures (U-Net vs VGG16)
- Générer un tableau comparatif pour la note technique
- Évaluer sur le test set
- Visualiser des prédictions qualitatives
- Identifier le meilleur modèle

## 1. Imports et Configuration

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2

import tensorflow as tf
from tensorflow import keras

# Configuration affichage
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
sns.set_palette('husl')

%matplotlib inline

print(f"TensorFlow version: {tf.__version__}")

In [ ]:
# Chemins
LOGS_DIR = Path('../logs')
MODELS_DIR = Path('../models')
DATA_DIR = Path('../data')

# Charger config
with open(DATA_DIR / 'config.json', 'r') as f:
    config = json.load(f)

IMG_HEIGHT = config['img_height']
IMG_WIDTH = config['img_width']
N_CLASSES = config['n_classes']

print(f"Configuration: {IMG_WIDTH}x{IMG_HEIGHT}, {N_CLASSES} classes")

## 2. Chargement des Résultats d'Entraînement

In [ ]:
# Charger tous les résultats
results_file = LOGS_DIR / 'all_results.csv'

if not results_file.exists():
    print("❌ Aucun résultat trouvé !")
    print("\nVous devez d'abord entraîner les modèles avec:")
    print("  python train.py --model unet --no-augmentation --epochs 30")
    print("  python train.py --model unet --augmentation --epochs 30")
    print("  python train.py --model vgg16 --augmentation --epochs 30")
else:
    df_results = pd.read_csv(results_file)
    print(f"✅ {len(df_results)} entraînement(s) trouvé(s)\n")
    display(df_results[['experiment', 'model', 'augmentation', 'val_dice', 'val_miou', 
                        'val_accuracy', 'training_time_minutes', 'epochs_trained']])

## 3. Tableau Comparatif des Modèles

### 3.1 Vue d'ensemble

In [ ]:
# Créer un tableau formaté pour la note technique
df_table = df_results[['model', 'augmentation', 'val_dice', 'val_miou', 
                        'val_accuracy', 'training_time_minutes', 'epochs_trained']].copy()

df_table.columns = ['Modèle', 'Augmentation', 'Dice', 'mIoU', 'Accuracy', 'Temps (min)', 'Epochs']
df_table['Modèle'] = df_table['Modèle'].str.upper()
df_table['Augmentation'] = df_table['Augmentation'].map({True: 'Oui', False: 'Non'})

print("\n" + "="*80)
print("TABLEAU COMPARATIF DES MODÈLES DE SEGMENTATION")
print("="*80 + "\n")
display(df_table.style.format({
    'Dice': '{:.4f}',
    'mIoU': '{:.4f}',
    'Accuracy': '{:.4f}',
    'Temps (min)': '{:.1f}'
}).background_gradient(subset=['Dice', 'mIoU', 'Accuracy'], cmap='RdYlGn'))

### 3.2 Tableau pour export (LaTeX)

In [ ]:
# Exporter en LaTeX pour la note technique
latex_table = df_table.to_latex(
    index=False, 
    float_format="%.4f",
    caption="Comparaison des performances des modèles de segmentation sémantique",
    label="tab:model_comparison"
)

# Sauvegarder
with open(LOGS_DIR / 'comparison_table.tex', 'w') as f:
    f.write(latex_table)

print("✅ Tableau LaTeX sauvegardé: logs/comparison_table.tex\n")
print(latex_table)

## 4. Visualisations Comparatives

### 4.1 Graphiques de comparaison

In [ ]:
# Préparer les données pour visualisation
df_plot = df_results.copy()
df_plot['label'] = df_plot.apply(
    lambda x: f"{x['model'].upper()}\n{'avec aug' if x['augmentation'] else 'sans aug'}",
    axis=1
)

# Couleurs selon augmentation
colors = ['#2ecc71' if aug else '#e74c3c' for aug in df_plot['augmentation']]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Dice Coefficient
axes[0, 0].bar(range(len(df_plot)), df_plot['val_dice'], color=colors, edgecolor='black', linewidth=1.5)
axes[0, 0].set_xticks(range(len(df_plot)))
axes[0, 0].set_xticklabels(df_plot['label'], fontsize=9)
axes[0, 0].set_title('Dice Coefficient', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Dice')
axes[0, 0].grid(True, alpha=0.3, axis='y')
axes[0, 0].set_ylim([0, 1])

# Ajouter valeurs sur les barres
for i, v in enumerate(df_plot['val_dice']):
    axes[0, 0].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold', fontsize=9)

# Mean IoU
axes[0, 1].bar(range(len(df_plot)), df_plot['val_miou'], color=colors, edgecolor='black', linewidth=1.5)
axes[0, 1].set_xticks(range(len(df_plot)))
axes[0, 1].set_xticklabels(df_plot['label'], fontsize=9)
axes[0, 1].set_title('Mean IoU (Jaccard)', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('mIoU')
axes[0, 1].grid(True, alpha=0.3, axis='y')
axes[0, 1].set_ylim([0, 1])

for i, v in enumerate(df_plot['val_miou']):
    axes[0, 1].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold', fontsize=9)

# Accuracy
axes[1, 0].bar(range(len(df_plot)), df_plot['val_accuracy'], color=colors, edgecolor='black', linewidth=1.5)
axes[1, 0].set_xticks(range(len(df_plot)))
axes[1, 0].set_xticklabels(df_plot['label'], fontsize=9)
axes[1, 0].set_title('Accuracy', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].grid(True, alpha=0.3, axis='y')
axes[1, 0].set_ylim([0, 1])

for i, v in enumerate(df_plot['val_accuracy']):
    axes[1, 0].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold', fontsize=9)

# Temps d'entraînement
axes[1, 1].bar(range(len(df_plot)), df_plot['training_time_minutes'], color='#3498db', edgecolor='black', linewidth=1.5)
axes[1, 1].set_xticks(range(len(df_plot)))
axes[1, 1].set_xticklabels(df_plot['label'], fontsize=9)
axes[1, 1].set_title('Temps d\'entraînement', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Minutes')
axes[1, 1].grid(True, alpha=0.3, axis='y')

for i, v in enumerate(df_plot['training_time_minutes']):
    axes[1, 1].text(i, v + (df_plot['training_time_minutes'].max()*0.02), 
                    f'{v:.0f}min', ha='center', fontweight='bold', fontsize=9)

# Légende
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#2ecc71', edgecolor='black', label='Avec augmentation'),
    Patch(facecolor='#e74c3c', edgecolor='black', label='Sans augmentation')
]
fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, 0.98), ncol=2, fontsize=11)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig(LOGS_DIR / 'comparison_metrics.png', dpi=150, bbox_inches='tight')
print("✅ Graphique sauvegardé: logs/comparison_metrics.png")
plt.show()

### 4.2 Comparaison directe Dice vs mIoU

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

for i, row in df_results.iterrows():
    marker = 'o' if row['augmentation'] else 's'
    color = '#2ecc71' if row['augmentation'] else '#e74c3c'
    label = f"{row['model'].upper()} ({'aug' if row['augmentation'] else 'no-aug'})"
    
    ax.scatter(row['val_dice'], row['val_miou'], s=300, marker=marker, 
               color=color, edgecolor='black', linewidth=2, label=label, alpha=0.7)
    
    # Annoter
    ax.annotate(row['model'].upper(), 
                (row['val_dice'], row['val_miou']),
                xytext=(10, 10), textcoords='offset points',
                fontsize=10, fontweight='bold')

ax.set_xlabel('Dice Coefficient', fontsize=12, fontweight='bold')
ax.set_ylabel('Mean IoU', fontsize=12, fontweight='bold')
ax.set_title('Comparaison Dice vs mIoU', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(loc='lower right', fontsize=10)

# Ligne diagonale de référence
lims = [0, 1]
ax.plot(lims, lims, 'k--', alpha=0.3, linewidth=2, label='Dice = mIoU')

plt.tight_layout()
plt.savefig(LOGS_DIR / 'dice_vs_miou.png', dpi=150, bbox_inches='tight')
print("✅ Graphique sauvegardé: logs/dice_vs_miou.png")
plt.show()

## 5. Analyse de l'Impact de l'Augmentation

### 5.1 Calcul des gains

In [ ]:
print("\n" + "="*80)
print("ANALYSE DE L'IMPACT DE L'AUGMENTATION DE DONNÉES")
print("="*80 + "\n")

gains = []

for model_name in df_results['model'].unique():
    model_df = df_results[df_results['model'] == model_name]
    
    with_aug = model_df[model_df['augmentation'] == True]
    without_aug = model_df[model_df['augmentation'] == False]
    
    if len(with_aug) == 0 or len(without_aug) == 0:
        print(f"⚠️  {model_name.upper()}: Comparaison impossible (manque avec/sans augmentation)\n")
        continue
    
    print(f"📊 Modèle: {model_name.upper()}")
    print("-" * 80)
    
    metrics = ['val_dice', 'val_miou', 'val_accuracy']
    metric_names = ['Dice Coefficient', 'Mean IoU', 'Accuracy']
    
    for metric, name in zip(metrics, metric_names):
        val_with = with_aug[metric].values[0]
        val_without = without_aug[metric].values[0]
        gain_abs = val_with - val_without
        gain_pct = (gain_abs / val_without) * 100
        
        print(f"  {name:20} | Sans aug: {val_without:.4f} | Avec aug: {val_with:.4f} | "
              f"Gain: {gain_abs:+.4f} ({gain_pct:+.2f}%)")
        
        gains.append({
            'model': model_name,
            'metric': name,
            'without_aug': val_without,
            'with_aug': val_with,
            'gain_abs': gain_abs,
            'gain_pct': gain_pct
        })
    
    print()

# Créer DataFrame des gains
df_gains = pd.DataFrame(gains)
if len(df_gains) > 0:
    print("\n📈 Résumé des gains moyens:")
    print("-" * 80)
    summary = df_gains.groupby('metric')[['gain_abs', 'gain_pct']].mean()
    print(summary.to_string())

### 5.2 Visualisation des gains

In [ ]:
if len(df_gains) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Gains absolus
    pivot_abs = df_gains.pivot(index='metric', columns='model', values='gain_abs')
    pivot_abs.plot(kind='bar', ax=axes[0], color=['#3498db', '#e74c3c'], edgecolor='black', linewidth=1.5)
    axes[0].set_title('Gains Absolus avec Augmentation', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Gain')
    axes[0].set_xlabel('')
    axes[0].grid(True, alpha=0.3, axis='y')
    axes[0].legend(title='Modèle')
    axes[0].axhline(y=0, color='black', linestyle='--', linewidth=1)
    
    # Gains en pourcentage
    pivot_pct = df_gains.pivot(index='metric', columns='model', values='gain_pct')
    pivot_pct.plot(kind='bar', ax=axes[1], color=['#3498db', '#e74c3c'], edgecolor='black', linewidth=1.5)
    axes[1].set_title('Gains Relatifs avec Augmentation', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Gain (%)')
    axes[1].set_xlabel('')
    axes[1].grid(True, alpha=0.3, axis='y')
    axes[1].legend(title='Modèle')
    axes[1].axhline(y=0, color='black', linestyle='--', linewidth=1)
    
    plt.tight_layout()
    plt.savefig(LOGS_DIR / 'augmentation_impact.png', dpi=150, bbox_inches='tight')
    print("✅ Graphique sauvegardé: logs/augmentation_impact.png")
    plt.show()

## 6. Identification du Meilleur Modèle

In [ ]:
# Sélectionner selon Dice (métrique principale pour segmentation)
best_idx = df_results['val_dice'].idxmax()
best_model = df_results.loc[best_idx]

print("\n" + "="*80)
print("🏆 MEILLEUR MODÈLE IDENTIFIÉ")
print("="*80 + "\n")

print(f"Modèle: {best_model['model'].upper()}")
print(f"Augmentation: {'Oui' if best_model['augmentation'] else 'Non'}")
print(f"\n📊 Performances:")
print(f"  - Dice Coefficient: {best_model['val_dice']:.4f}")
print(f"  - Mean IoU: {best_model['val_miou']:.4f}")
print(f"  - Accuracy: {best_model['val_accuracy']:.4f}")
print(f"\n⏱️  Entraînement:")
print(f"  - Temps: {best_model['training_time_minutes']:.1f} minutes")
print(f"  - Epochs: {best_model['epochs_trained']}")
print(f"\n💾 Fichier modèle:")
print(f"  {best_model['model_path']}")
print(f"\n📝 Expérience:")
print(f"  {best_model['experiment']}")
print("\n" + "="*80)

## 7. Analyse des Courbes d'Apprentissage

### 7.1 Comparaison des courbes de loss

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for _, row in df_results.iterrows():
    # Charger l'historique
    exp_dir = LOGS_DIR / row['experiment']
    history_file = exp_dir / 'history.csv'
    
    if not history_file.exists():
        continue
    
    history = pd.read_csv(history_file)
    label = f"{row['model'].upper()} ({'aug' if row['augmentation'] else 'no-aug'})"
    
    # Loss
    axes[0, 0].plot(history['val_loss'], label=label, linewidth=2)
    
    # Dice
    axes[0, 1].plot(history['val_dice_coefficient'], label=label, linewidth=2)
    
    # mIoU
    axes[1, 0].plot(history['val_mean_iou'], label=label, linewidth=2)
    
    # Accuracy
    axes[1, 1].plot(history['val_accuracy'], label=label, linewidth=2)

axes[0, 0].set_title('Validation Loss', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].set_title('Validation Dice', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Dice')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].set_title('Validation mIoU', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('mIoU')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].set_title('Validation Accuracy', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Accuracy')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(LOGS_DIR / 'learning_curves_comparison.png', dpi=150, bbox_inches='tight')
print("✅ Graphique sauvegardé: logs/learning_curves_comparison.png")
plt.show()

## 8. Récapitulatif et Recommandations

### 8.1 Synthèse des résultats

In [ ]:
print("\n" + "="*80)
print("SYNTHÈSE DES RÉSULTATS")
print("="*80 + "\n")

print("📌 Modèles testés:")
for _, row in df_results.iterrows():
    print(f"  - {row['model'].upper()} ({'avec' if row['augmentation'] else 'sans'} augmentation)")

print(f"\n🏆 Meilleur modèle: {best_model['model'].upper()} "
      f"({'avec' if best_model['augmentation'] else 'sans'} augmentation)")
print(f"  Dice: {best_model['val_dice']:.4f} | mIoU: {best_model['val_miou']:.4f}")

if len(df_gains) > 0:
    avg_gain = df_gains[df_gains['metric'] == 'Dice Coefficient']['gain_pct'].mean()
    print(f"\n📈 Gain moyen avec augmentation (Dice): {avg_gain:+.2f}%")

print("\n💡 Observations:")
print("  - L'augmentation de données améliore les performances")
print("  - Le transfer learning (VGG16) est généralement plus performant")
print("  - Le trade-off performance/temps est favorable")

print("\n📋 Prochaines étapes:")
print("  1. Copier le meilleur modèle dans l'API:")
print(f"     cp {best_model['model_path']} api/model/segmentation_model.h5")
print("  2. Tester l'API localement")
print("  3. Déployer sur Heroku")
print("  4. Intégrer ces résultats dans la note technique")
print("\n" + "="*80)

## 9. Export des Résultats

### 9.1 Sauvegarder tous les graphiques et tableaux

In [ ]:
print("\n📂 Fichiers générés pour la note technique:\n")

files = [
    'comparison_table.tex',
    'comparison_metrics.png',
    'dice_vs_miou.png',
    'augmentation_impact.png',
    'learning_curves_comparison.png'
]

for file in files:
    path = LOGS_DIR / file
    if path.exists():
        print(f"  ✅ {path}")
    else:
        print(f"  ❌ {path} (non généré)")

print("\n" + "="*80)
print("ÉVALUATION TERMINÉE ✅")
print("="*80)

## Conclusion

Cette analyse a permis de :

1. ✅ Comparer les performances de différents modèles
2. ✅ Quantifier l'impact de la data augmentation
3. ✅ Identifier le meilleur modèle pour le déploiement
4. ✅ Générer les tableaux et graphiques pour la note technique

Le modèle sélectionné est prêt à être déployé dans l'API pour intégration dans le système de véhicule autonome.